# CS 412 - Machine Learning - Term Project

Barış Hancili - 27896

Elif Gödüş - 29241

---


**Objective**

The objective of this project is to predict student grades based on interactions with ChatGPT, as well as implementing and analysing regression algorithms to find best performer for this example


---



**Dataset**

The dataset consists of two main components:


**ChatGPT Interactions:** These interactions are stored as HTML files, each containing a conversation between a student and ChatGPT.

**Student Grades:** This information is stored in a CSV file, where each row corresponds to a student along with their grade.


---



**Methodology**

**Data Preprocessing:** The HTML files are processed to extract the conversational data, and the CSV file is used to link the conversations with the corresponding student grades.

**Feature Engineering:** Features such as the length of conversations, sentiment analysis of the interactions, and other relevant metrics are derived from the conversational data.

**Model Building:** Regression algorithms such as Linear Regression, Decision Trees, Random Forest, and potentially Neural Network Regression are implemented to predict student grades based on the derived features.

**Model Evaluation:** The models are evaluated using metrics such as Mean Squared Error (MSE) and R-squared to assess their performance in predicting student grades.

**Importing Necessary Libraries**

In [702]:
import os
import re
import tqdm
from glob import glob
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import graphviz

from collections import defaultdict
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Extracting Conversation**

Following code uses the glob module to find all HTML files in a specified directory (**dataset**), then iterates through each file. For each file, it reads its content using open and read, and then parses the HTML content using BeautifulSoup. It looks for specific div elements within the HTML that likely contain conversation data and extracts relevant information such as the role of the message author and the message text. This information is then organized into a dictionary where the keys are unique identifiers for each file, and the values are lists of dictionaries containing the extracted conversation data.

In [703]:
data_path = "dataset/*.html"

code2convos = dict()

pbar = tqdm.tqdm(sorted(list(glob(data_path))))
for path in pbar:
    # print(Path.cwd() / path)
    file_code = os.path.basename(path).split(".")[0]
    with open(path, "r", encoding="latin1") as fh:

        # get the file id to use it as key later on
        fid = os.path.basename(path).split(".")[0]

        # read the html file
        html_page = fh.read()

        # parse the html file with bs4 so we can extract needed stuff
        soup = BeautifulSoup(html_page, "html.parser")

        # grab the conversations with the data-testid pattern
        data_test_id_pattern = re.compile(r"conversation-turn-[0-9]+")
        conversations = soup.find_all("div", attrs={"data-testid": data_test_id_pattern})

        convo_texts = []

        for i, convo in enumerate(conversations):
            convo = convo.find_all("div", attrs={"data-message-author-role":re.compile( r"[user|assistant]") })
            if len(convo) > 0:
                role = convo[0].get("data-message-author-role")
                convo_texts.append({
                        "role" : role,
                        "text" : convo[0].text
                    }
                )

        code2convos[file_code] = convo_texts

100%|██████████| 127/127 [00:24<00:00,  5.10it/s]


**Example Conversation:**




In [704]:
pprint(code2convos["0031c86e-81f4-4eef-9e0e-28037abf9883"][0])

{'role': 'user',
 'text': 'Load a CSV file into a Pandas in Python. The file is named '
         "'cs412_hw1_dataset.csv' and contains columns like 'Species', "
         "'Island', 'Sex', 'Diet', 'Year', 'Life Stage', 'Body Mass (g)', "
         "'Bill Length (mm)', 'Bill Depth (mm)', 'Flipper Length (mm)', and "
         "'Health Metrics'. \n"}


### **Text Preprocessing**

In our project, text preprocessing played a crucial role in preparing the data for analysis. By applying lowercasing, removing punctuation and special characters, tokenization, stop-words removal, stemming, and text normalization to both the conversations and questions, we aimed to standardize the textual data and make it more amenable to the algorithms.

**Lowercasing**

In [705]:
for file_code, convos in code2convos.items():
    for convo in convos:
        convo['text'] = convo['text'].lower()

**Removing Punctuation and Special Characters**

In [706]:
for file_code, convos in code2convos.items():
    for convo in convos:
        convo['text'] = re.sub(r'[^a-zA-Z0-9\s]', '', convo['text'])

**Tokenization**

In [707]:
from nltk.tokenize import word_tokenize

for file_code, convos in code2convos.items():
    for convo in convos:
        tokens = word_tokenize(convo['text'])
        convo['tokens'] = tokens

**Stop-Words Removal**

In [708]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

for file_code, convos in code2convos.items():
    for convo in convos:
        filtered_tokens = [word for word in convo['tokens'] if word.lower() not in stop_words]
        convo['filtered_tokens'] = filtered_tokens

**Stemming**

In [709]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer(language='english')

for file_code, convos in code2convos.items():
    for convo in convos:
        stemmed_tokens = [stemmer.stem(word) for word in convo['filtered_tokens']]
        convo['stemmed_tokens'] = stemmed_tokens

**Text Normalization**

In [710]:
for file_code, convos in code2convos.items():
    for convo in convos:
        text = ' '.join(convo['stemmed_tokens'])
        convo['text'] = text
        keys_to_remove = [key for key in convo.keys() if key not in ['role', 'text']]
        for key in keys_to_remove:
            del convo[key]

**Example Conversation After Preprocessing:**

In [711]:
pprint(code2convos["0031c86e-81f4-4eef-9e0e-28037abf9883"][0])

{'role': 'user',
 'text': 'load csv file panda python file name cs412hw1datasetcsv contain '
         'column like speci island sex diet year life stage bodi mass g bill '
         'length mm bill depth mm flipper length mm health metric'}


**Preprocessing for Questions**

In [712]:
questions = [
    """Initialize
*   First make a copy of the notebook given to you as a starter.
*   Make sure you choose Connect form upper right.
*   You may upload the data to the section on your left on Colab, than right click on the .csv file and get the path of the file by clicking on "Copy Path". You will be using it when loading the data.

""",
#####################
    """Load training dataset (5 pts)
    *  Read the .csv file with the pandas library
""",
#####################
"""Understanding the dataset & Preprocessing (15 pts)
Understanding the Dataset: (5 pts)
> - Find the shape of the dataset (number of samples & number of attributes). (Hint: You can use the **shape** function)
> - Display variable names (both dependent and independent).
> - Display the summary of the dataset. (Hint: You can use the **info** function)
> - Display the first 5 rows from training dataset. (Hint: You can use the **head** function)
Preprocessing: (10 pts)

> - Check if there are any missing values in the dataset. If there are, you can either drop these values or fill it with most common values in corresponding rows. **Be careful that you have enough data for training the  model.**

> - Encode categorical labels with the mappings given in the cell below. (Hint: You can use **map** function)
""",
"""Set X & y, split data (5 pts)

*   Shuffle the dataset.
*   Seperate your dependent variable X, and your independent variable y. The column health_metrics is y, the rest is X.
*   Split training and test sets as 80% and 20%, respectively.
""",
#####################
"""Features and Correlations (10 pts)

* Correlations of features with health (4 points)
Calculate the correlations for all features in dataset. Highlight any strong correlations with the target variable. Plot your results in a heatmap.

* Feature Selection (3 points)
Select a subset of features that are likely strong predictors, justifying your choices based on the computed correlations.

* Hypothetical Driver Features (3 points)
Propose two hypothetical features that could enhance the model's predictive accuracy for Y, explaining how they might be derived and their expected impact. Show the resulting correlations with target variable.

* __Note:__ You get can get help from GPT.
""",
#####################
"""Tune Hyperparameters (20 pts)
* Choose 2 hyperparameters to tune. You can use the Scikit learn decision tree documentation for the available hyperparameters *(Hyperparameters are listed under "Parameters" in the documentation)*. Use GridSearchCV for hyperparameter tuning, with a cross-validation value of 5. Use validation accuracy to pick the best hyper-parameter values. (15 pts)
-Explain the hyperparameters you chose to tune. *(What are the hyperparameters you chose? Why did you choose them?)* (5 pts)
""",
#####################
"""Re-train and plot the decision tree with the hyperparameters you have chosen (15 pts)
- Re-train model with the hyperparameters you have chosen in part 5). (10 pts)
- Plot the tree you have trained. (5 pts)
Hint: You can import the **plot_tree** function from the sklearn library.
""",
#####################
"""Test your classifier on the test set (20 pts)
- Predict the labels of testing data using the tree you have trained in step 6. (10 pts)
- Report the classification accuracy. (2 pts)
- Plot & investigate the confusion matrix. Fill the following blanks. (8 pts)
> The model most frequently mistakes class(es) _________ for class(es) _________.
Hint: You can use the confusion_matrix function from sklearn.metrics
""",
#####################
"""Find the information gain on the first split (10 pts)""",
#####################
]

In [713]:
stemmer = SnowballStemmer(language='english')
stop_words = set(stopwords.words('english'))

# Preprocess each question in the list
for i in range(len(questions)):
    # Lowercasing
    questions[i] = questions[i].lower()
    # Removing Punctuation and Special Characters
    questions[i] = re.sub(r'[^a-zA-Z0-9\s]', '', questions[i])
    # Tokenization
    tokens = word_tokenize(questions[i])
    # Stop-Words Removal
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    # Stemming
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    # Text Normalization
    questions[i] = ' '.join(stemmed_tokens)

**Example Question After Preprocessing:**

In [714]:
questions[4]

'featur correl 10 pts correl featur health 4 point calcul correl featur dataset highlight strong correl target variabl plot result heatmap featur select 3 point select subset featur like strong predictor justifi choic base comput correl hypothet driver featur 3 point propos two hypothet featur could enhanc model predict accuraci explain might deriv expect impact show result correl target variabl note get get help gpt'

### **Prompt Matching**

In our project, we have implemented a process for matching prompts with questions, aiming to establish correlations between user prompts and specific questions. This matching process involves techniques such as comparing prompt similarity to questions using methods like cosine similarity.

In [715]:
prompts = []
code2prompts = defaultdict(list)
for code , convos in code2convos.items():
    user_prompts = []
    for conv in convos:
        if conv["role"] == "user":
            prompts.append(conv["text"])
            user_prompts.append(conv["text"])
    code2prompts[code] = user_prompts

In [716]:
vectorizer = TfidfVectorizer()
vectorizer = vectorizer.fit(prompts + questions)

In [717]:
questions_TF_IDF = pd.DataFrame(vectorizer.transform(questions).toarray(), columns=vectorizer.get_feature_names_out())
questions_TF_IDF.head(2)

,00,0000196,0000282,0000284,0000360,0000469,0000750,0000886,0000991,0001,...,ytrainxtrainrootsplitindex,ytreeapplyx,yvalu,zaman,zdm,zero,zerodivisionerror,zeroth,zipclassnam,zipnpuniquechildindicesaftersplit
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [718]:
code2prompts_tf_idf = dict()
for code, user_prompts in code2prompts.items():
    if len(user_prompts) == 0:
        print(code+".html")
        continue
    prompts_TF_IDF = pd.DataFrame(vectorizer.transform(user_prompts).toarray(), columns=vectorizer.get_feature_names_out())
    code2prompts_tf_idf[code] = prompts_TF_IDF

139235c7-736c-4237-92f0-92e8c116832c.html
668ad17e-0240-49f7-b5a7-d22e502554c6.html
b0640e51-6879-40cb-a4f5-329f952ef99d.html
da6b70d5-29f6-491a-ad46-037c77067128.html


In [719]:
code2cosine = dict()
for code, user_prompts_tf_idf in code2prompts_tf_idf.items():
    code2cosine[code] = pd.DataFrame(cosine_similarity(questions_TF_IDF,user_prompts_tf_idf))

In [720]:
code2questionmapping = dict()
for code, cosine_scores in code2cosine.items():
    code2questionmapping[code] = code2cosine[code].max(axis=1).tolist()

question_mapping_scores = pd.DataFrame(code2questionmapping).T
question_mapping_scores.reset_index(inplace=True)
question_mapping_scores.rename(columns={i: f"Q_{i}" for i in range(len(questions))}, inplace=True)
question_mapping_scores.rename(columns={"index" : "code"}, inplace=True)

**Handling Edge Cases**

In our project, handling edge cases is crucial for ensuring the robustness and accuracy of our machine learning model. These edge cases, such as instances where a student has never used ChatGPT for a specific question resulting in a conversation value of **0.0**, or where a student has copy-pasted a question leading to a high similarity score of **1.0**, require special attention. To address this, we have implemented a strategy to replace these extreme values with the mean of the respective column. By doing so, we aim to create a more balanced and representative dataset.

In [721]:
import numpy as np

for column in question_mapping_scores.columns:
    # Convert the column to numeric
    numeric_column = pd.to_numeric(question_mapping_scores[column], errors='coerce')

    # Calculate the average of the numeric values in the column
    column_average = np.nanmean(numeric_column)

    # Replace values higher than 1.0 with the column average
    question_mapping_scores[column] = np.where(numeric_column >= 1.0, column_average, question_mapping_scores[column])

<ipython-input-721-1404a2b0353d>:8: RuntimeWarning: Mean of empty slice
  column_average = np.nanmean(numeric_column)


In [722]:
import numpy as np

# Replace 0.0 with NaN for easier handling
question_mapping_scores.replace(0.0, np.nan, inplace=True)

# Calculate the column-wise average (ignoring NaN values)
column_averages = question_mapping_scores.mean()

# Replace NaN values with the calculated averages
question_mapping_scores.fillna(column_averages, inplace=True)

# Replace NaN with 0.0 if there are any remaining NaN values
question_mapping_scores.fillna(0.0, inplace=True)

<ipython-input-722-ee013d67baad>:7: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_averages = question_mapping_scores.mean()


**Resulted Ratio Table**

In [723]:
question_mapping_scores.head(5)

,code,Q_0,Q_1,Q_2,Q_3,Q_4,Q_5,Q_6,Q_7,Q_8
0,0031c86e-81f4-4eef-9e0e-28037abf9883,0.111065,0.281578,0.618961,0.432243,0.510732,0.551559,0.139785,0.181310,0.123534
1,0225686d-b825-4cac-8691-3a3a5343df2b,0.164216,0.802719,0.756955,0.849414,0.614525,0.989534,0.852307,0.636646,0.578007
2,041f950b-c013-409a-a642-cffff60b9d4b,0.089521,0.247172,0.502018,0.329575,0.627776,0.399371,0.445345,0.481055,0.268303
3,04f91058-d0f8-4324-83b2-19c671f433dc,0.059109,0.177408,0.171692,0.394055,0.519166,0.222054,0.129287,0.240920,0.361381
4,089eb66d-4c3a-4f58-b98f-a3774a2efb34,0.357090,0.618947,0.770297,0.659338,0.745080,0.924457,0.601452,0.937321,0.499306


### **Feature Engineering**

Our feature engineering process involves the creation of several key features essential for our machine learning model. These features include the Number of Prompts User Asked, which reflects user engagement, the Average Length of User and Assistant Prompts, providing insight into the conversational dynamics, and the Ratio of Words Between User and Assistant, indicating the balance of interaction. Additionally, we conduct Sentiment Analysis to categorize prompts as positive, negative, or neutral, capturing emotional context. Moreover, we calculate the Number of Error Prompts from User to understand potential challenges faced. Finally, we consider the Conversation Length to gauge overall interaction duration.

**Number of Prompts User Asked**

In [724]:
question_mapping_scores['#UserPrompts'] = [len(code2prompts[code]) for code in question_mapping_scores['code']]

**Average Length of User and Assistant Prompts**

In [725]:
# Calculate User Prompts Average Number of Characters
user_prompts_avg_chars = []
for code in question_mapping_scores['code']:
    if code in code2prompts:  # Check if there are user prompts for the code
        user_prompts = code2prompts[code]
        user_prompts_avg_chars.append(sum(len(prompt) for prompt in user_prompts) / len(user_prompts))
    else:
        user_prompts_avg_chars.append(0)  # If there are no user prompts, set the average to 0

# Calculate Assistant Prompts Average Number of Characters
assistant_prompts_avg_chars = []
for code in question_mapping_scores['code']:
    if code in code2convos:  # Check if there are conversations for the code
        convos = code2convos[code]
        assistant_prompts = [conv['text'] for conv in convos if conv['role'] != 'user']
        if assistant_prompts:  # Check if there are assistant prompts for the code
            assistant_prompts_avg_chars.append(sum(len(prompt) for prompt in assistant_prompts) / len(assistant_prompts))
        else:
            assistant_prompts_avg_chars.append(0)  # If there are no assistant prompts, set the average to 0
    else:
        assistant_prompts_avg_chars.append(0)  # If there are no conversations for the code, set the average to 0

question_mapping_scores['User_Prompts_Avg_Num_Chars'] = user_prompts_avg_chars
question_mapping_scores['Assistant_Prompts_Avg_Num_Chars'] = assistant_prompts_avg_chars

**Ratio of Words Between User and Assistant**

In [726]:
# Calculate Ratio of Words between User and Assistant
ratio_user_to_assistant_words = []
for code in question_mapping_scores['code']:
    if code in code2prompts and code in code2convos:  # Check if there are user prompts and conversations for the code
        user_words_count = sum(len(prompt.split()) for prompt in code2prompts[code])
        assistant_words_count = sum(len(conv['text'].split()) for conv in code2convos[code] if conv['role'] != 'user')
        if assistant_words_count == 0:  # Avoid division by zero
            ratio_user_to_assistant_words.append(0)
        else:
            ratio_user_to_assistant_words.append(user_words_count / assistant_words_count)
    else:
        ratio_user_to_assistant_words.append(0)  # If there are no user prompts or conversations for the code, set the ratio to 0

question_mapping_scores['User Assistant Ratio'] = ratio_user_to_assistant_words

**Sentiment Analysis**

In [727]:
from textblob import TextBlob

# Function to classify sentiment of a text
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

# Calculate sentiment for each conversation and count positive, negative, and neutral prompts
sentiment_counts = {'positive': [], 'negative': [], 'neutral': []}
for code in question_mapping_scores['code']:
    if code in code2prompts and code in code2convos:  # Check if there are user prompts and conversations for the code
        positive_count = 0
        negative_count = 0
        neutral_count = 0
        for conv in code2convos[code]:
            if conv['role'] == 'user':
                sentiment = get_sentiment(conv['text'])
                if sentiment == 'positive':
                    positive_count += 1
                elif sentiment == 'negative':
                    negative_count += 1
                else:
                    neutral_count += 1
        sentiment_counts['positive'].append(positive_count)
        sentiment_counts['negative'].append(negative_count)
        sentiment_counts['neutral'].append(neutral_count)
    else:
        sentiment_counts['positive'].append(0)
        sentiment_counts['negative'].append(0)
        sentiment_counts['neutral'].append(0)

question_mapping_scores['Positive'] = sentiment_counts['positive']
question_mapping_scores['Negative'] = sentiment_counts['negative']
question_mapping_scores['Neutral'] = sentiment_counts['neutral']

**Number of Error Prompts from User**

In [728]:
def contains_error(word):
    return "error" in word.lower() or "exception" in word.lower() or "typeerror" in word.lower()

def prompt_contains_error(prompt):
    return any(contains_error(word) for word in prompt.split())

# Calculate Ratio of User Prompts with at least one Error Word
ratio_user_prompts_with_error = []
for code in question_mapping_scores['code']:
    if code in code2prompts and code in code2convos:  # Check if there are user prompts and conversations for the code
        user_prompts = [prompt['text'] for prompt in code2convos[code] if prompt['role'] == 'user']
        user_prompts_with_error_count = sum(prompt_contains_error(prompt) for prompt in user_prompts)
        total_user_prompts_count = len(user_prompts)
        if total_user_prompts_count == 0:  # Avoid division by zero
            ratio_user_prompts_with_error.append(0)
        else:
            ratio_user_prompts_with_error.append(user_prompts_with_error_count / total_user_prompts_count)
    else:
        ratio_user_prompts_with_error.append(0)  # If there are no user prompts or conversations for the code, set the ratio to 0

question_mapping_scores['Error Ratio'] = ratio_user_prompts_with_error

**Conversation Length**

In [729]:
# Calculate Conversation Length in Terms of Total Words
conversation_lengths = []
for code in question_mapping_scores['code']:
    if code in code2convos:  # Check if there are conversations for the code
        total_words_count = sum(len(conv['text'].split()) for conv in code2convos[code])
        conversation_lengths.append(total_words_count)
    else:
        conversation_lengths.append(0)  # If there are no conversations for the code, set the length to 0

question_mapping_scores['Length'] = conversation_lengths

**Merging with Grades**

In [730]:
scores_df = pd.read_csv('scores.csv', index_col=None)

merged_df = pd.merge(question_mapping_scores, scores_df, on='code', how="left")
merged_df = merged_df.drop('Unnamed: 0', axis=1)

df = merged_df.drop_duplicates(keep='first')

**Resulted Data Frame**

In [731]:
df.head()

,code,Q_0,Q_1,Q_2,Q_3,Q_4,Q_5,Q_6,Q_7,Q_8,#UserPrompts,User_Prompts_Avg_Num_Chars,Assistant_Prompts_Avg_Num_Chars,User Assistant Ratio,Positive,Negative,Neutral,Error Ratio,Length,grade
0,0031c86e-81f4-4eef-9e0e-28037abf9883,0.111065,0.281578,0.618961,0.432243,0.510732,0.551559,0.139785,0.181310,0.123534,14,244.500000,1401.071429,0.212783,6,4,4,0.357143,2998,48.0
1,0225686d-b825-4cac-8691-3a3a5343df2b,0.164216,0.802719,0.756955,0.849414,0.614525,0.989534,0.852307,0.636646,0.578007,18,156.277778,1073.000000,0.182164,8,2,8,0.000000,2797,99.0
2,041f950b-c013-409a-a642-cffff60b9d4b,0.089521,0.247172,0.502018,0.329575,0.627776,0.399371,0.445345,0.481055,0.268303,9,417.444444,1101.555556,0.480653,3,0,6,0.555556,1722,90.0
3,04f91058-d0f8-4324-83b2-19c671f433dc,0.059109,0.177408,0.171692,0.394055,0.519166,0.222054,0.129287,0.240920,0.361381,20,65.850000,1112.500000,0.071978,4,2,14,0.050000,3068,97.0
4,089eb66d-4c3a-4f58-b98f-a3774a2efb34,0.357090,0.618947,0.770297,0.659338,0.745080,0.924457,0.601452,0.937321,0.499306,86,132.558140,934.360465,0.152105,31,12,43,0.081395,11248,100.0


**Detecting and Treating Outliers**

In our project, detecting and treating outliers is essential to ensure the reliability and accuracy of our predictive models. By identifying and replacing outliers with the average value of their respective node, we aim to mitigate their impact on our analysis. This process helps in creating a more representative and stable dataset, which in turn improves the robustness of our machine learning models and enhances the accuracy of our predictions.

In [732]:
import pandas as pd

# Dictionary to store acceptance regions and outlier indexes for each column
acceptance_regions = {}
outlier_indexes_dict = {}

# Loop through all columns except non-numeric columns
for col in df.select_dtypes(include='number').columns:
    # Calculate the mean and standard deviation of the column
    mean = df[col].mean()
    std = df[col].std()

    # Define the acceptance region based on mean and standard deviation
    lower_bound = mean - 2 * std
    upper_bound = mean + 2 * std

    # Find the indexes of rows that fall outside the acceptance region
    outlier_indexes = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index

    # Replace outliers with the column mean
    df.loc[outlier_indexes, col] = mean

    # Store the acceptance region and outlier indexes for the column
    acceptance_regions[col] = (lower_bound, upper_bound)
    outlier_indexes_dict[col] = outlier_indexes.tolist()

### **Prediction and Analysis**

During the prediction and analysis phase of our project, we followed a systematic approach to evaluate and compare the performance of various regression algorithms. Initially, we divided our dataset into training and testing subsets, allocating 80% for training and 20% for testing. Subsequently, we implemented **Linear Regression**, **Decision Tree**, **Random Forest**, and **k Nearest Neighbors (KNN)** algorithms. This comprehensive analysis allowed us to assess the strengths and weaknesses of each algorithm and determine the most suitable model for our predictive task.

**Splitting Data**

In [733]:
X = df[df.columns[1:-1]].to_numpy()
y = df["grade"].to_numpy()

In [734]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train set size:", len(X_train))
print("Test set size:", len(X_test))

Train set size: 99
Test set size: 25


**Drop Rows with NaN Values from Training Sets**

In [735]:
import numpy as np

# Check for NaN values in X_train
nan_indices_train = np.argwhere(np.isnan(X_train))
if len(nan_indices_train) > 0:
    print("NaN values found in X_train at indices:", nan_indices_train.flatten())
    # Drop rows with NaN values from X_train and y_train
    X_train = np.delete(X_train, nan_indices_train[:, 0], axis=0)
    y_train = np.delete(y_train, nan_indices_train[:, 0], axis=0)
    print("NaN values dropped from X_train and y_train.")

# Check for NaN values in X_test
nan_indices_test = np.argwhere(np.isnan(X_test))
if len(nan_indices_test) > 0:
    print("NaN values found in X_test at indices:", nan_indices_test.flatten())
    # Drop rows with NaN values from X_test and y_test
    X_test = np.delete(X_test, nan_indices_test[:, 0], axis=0)
    y_test = np.delete(y_test, nan_indices_test[:, 0], axis=0)
    print("NaN values dropped from X_test and y_test.")
else:
    print("No NaN values found in X_test.")


No NaN values found in X_test.


In [736]:
import numpy as np

# Check for NaN values in y_train
nan_indices_y_train = np.argwhere(np.isnan(y_train))
if len(nan_indices_y_train) > 0:
    print("NaN values found in y_train at indices:", nan_indices_y_train.flatten())
    # Drop rows with NaN values from X_train and y_train
    X_train = np.delete(X_train, nan_indices_y_train[:, 0], axis=0)
    y_train = np.delete(y_train, nan_indices_y_train[:, 0], axis=0)
    print("NaN values dropped from X_train and y_train.")

# Check for NaN values in y_test
nan_indices_y_test = np.argwhere(np.isnan(y_test))
if len(nan_indices_y_test) > 0:
    print("NaN values found in y_test at indices:", nan_indices_y_test.flatten())
    # Drop rows with NaN values from X_test and y_test
    X_test = np.delete(X_test, nan_indices_y_test[:, 0], axis=0)
    y_test = np.delete(y_test, nan_indices_y_test[:, 0], axis=0)
    print("NaN values dropped from X_test and y_test.")
else:
    print("No NaN values found in y_test.")


NaN values found in y_train at indices: [13]
NaN values dropped from X_train and y_train.
NaN values found in y_test at indices: [12]
NaN values dropped from X_test and y_test.


In [737]:
print("Updated Train set size:", len(X_train))
print("Updated Test set size:", len(X_test))

Updated Train set size: 98
Updated Test set size: 24


**Base Case**

The base case provided by the assistant serves as a benchmark against which we evaluate the performance of our regression algorithms. By comparing the predictions generated by our models with the base case, we can assess their accuracy and effectiveness in capturing the underlying patterns in the data.

In [738]:
regressor = DecisionTreeRegressor(random_state=0,criterion='squared_error', max_depth=10)
regressor.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=10, random_state=0)

In [739]:
extracted_MSEs = regressor.tree_.impurity
for idx, MSE in enumerate(regressor.tree_.impurity):
    print("Node {} has MSE {}".format(idx,MSE))

Node 0 has MSE 46.91066707206119
Node 1 has MSE 61.840000000001055
Node 2 has MSE 21.1875
Node 3 has MSE 2.25
Node 4 has MSE 0.0
Node 5 has MSE 0.0
Node 6 has MSE 4.0
Node 7 has MSE 0.0
Node 8 has MSE 0.0
Node 9 has MSE 0.0
Node 10 has MSE 36.72284966666666
Node 11 has MSE 41.47222222222172
Node 12 has MSE 6.25
Node 13 has MSE 0.0
Node 14 has MSE 0.0
Node 15 has MSE 2.75
Node 16 has MSE 1.0
Node 17 has MSE 0.0
Node 18 has MSE 0.0
Node 19 has MSE 0.0
Node 20 has MSE 29.028847529361883
Node 21 has MSE 14.325333543589295
Node 22 has MSE 11.711688367973693
Node 23 has MSE 12.88947443813231
Node 24 has MSE 23.1875
Node 25 has MSE 4.0
Node 26 has MSE 0.0
Node 27 has MSE 0.0
Node 28 has MSE 6.25
Node 29 has MSE 0.0
Node 30 has MSE 0.0
Node 31 has MSE 10.015632260308848
Node 32 has MSE 1.4320987654336932
Node 33 has MSE 0.2400000000016007
Node 34 has MSE 0.0
Node 35 has MSE 0.0
Node 36 has MSE 1.5
Node 37 has MSE 0.0
Node 38 has MSE 0.22222222222262644
Node 39 has MSE 0.0
Node 40 has MSE 0.0
N

In [740]:
# Prediction
y_train_pred = regressor.predict(X_train)
y_test_pred = regressor.predict(X_test)

# Calculation of Mean Squared Error (MSE)
print("MSE Train:", mean_squared_error(y_train,y_train_pred))
print("MSE TEST:", mean_squared_error(y_test,y_test_pred))

print("R2 Train:", r2_score(y_train,y_train_pred))
print("R2 TEST:", r2_score(y_test,y_test_pred))

MSE Train: 1.5807427128719962
MSE TEST: 108.83360526494333
R2 Train: 0.9663031286584819
R2 TEST: -3.6389058810635504


**Linear Regression**

In [741]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train, y_train)

# Training set predictions and evaluation
y_train_pred = model.predict(X_train)
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Test set predictions and evaluation
y_test_pred = model.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

print("MSE Train:", mse_train)
print("MSE TEST:", mse_test)
print("R2 Train:", r2_train)
print("R2 TEST:", r2_test)

MSE Train: 34.992067326024284
MSE TEST: 40.23351649840823
R2 Train: 0.25407013990501226
R2 TEST: -0.7149068603027549


**Decision Tree**

In [743]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

tree_model = DecisionTreeRegressor()
tree_model.fit(X_train, y_train)

# Training set predictions and evaluation
y_train_pred_tree = tree_model.predict(X_train)
mse_train_tree = mean_squared_error(y_train, y_train_pred_tree)
r2_train_tree = r2_score(y_train, y_train_pred_tree)

# Test set predictions and evaluation
y_test_pred_tree = tree_model.predict(X_test)
mse_test_tree = mean_squared_error(y_test, y_test_pred_tree)
r2_test_tree = r2_score(y_test, y_test_pred_tree)

print("Decision Tree - MSE Train:", mse_train_tree)
print("Decision Tree - MSE TEST:", mse_test_tree)
print("Decision Tree - R2 Train:", r2_train_tree)
print("Decision Tree - R2 TEST:", r2_test_tree)

Decision Tree - MSE Train: 0.00510204081632653
Decision Tree - MSE TEST: 109.92301296694438
Decision Tree - R2 Train: 0.999891239218396
Decision Tree - R2 TEST: -3.685340617681768


In [744]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

tree_model = DecisionTreeRegressor(random_state=0, splitter="random")
tree_model.fit(X_train, y_train)

# Training set predictions and evaluation
y_train_pred_tree = tree_model.predict(X_train)
mse_train_tree = mean_squared_error(y_train, y_train_pred_tree)
r2_train_tree = r2_score(y_train, y_train_pred_tree)

# Test set predictions and evaluation
y_test_pred_tree = tree_model.predict(X_test)
mse_test_tree = mean_squared_error(y_test, y_test_pred_tree)
r2_test_tree = r2_score(y_test, y_test_pred_tree)

print("Hyperparameter Decision Tree - MSE Train:", mse_train_tree)
print("Hyperparameter Decision Tree - MSE TEST:", mse_test_tree)
print("Hyperparameter Decision Tree - R2 Train:", r2_train_tree)
print("Hyperparameter Decision Tree - R2 TEST:", r2_test_tree)

Hyperparameter Decision Tree - MSE Train: 0.00510204081632653
Hyperparameter Decision Tree - MSE TEST: 41.862732912299556
Hyperparameter Decision Tree - R2 Train: 0.999891239218396
Hyperparameter Decision Tree - R2 TEST: -0.7843503156171974


**Random Forest**

In [765]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# Training set predictions and evaluation
y_train_pred_rf = rf_model.predict(X_train)
mse_train_rf = mean_squared_error(y_train, y_train_pred_rf)
r2_train_rf = r2_score(y_train, y_train_pred_rf)

# Test set predictions and evaluation
y_test_pred_rf = rf_model.predict(X_test)
mse_test_rf = mean_squared_error(y_test, y_test_pred_rf)
r2_test_rf = r2_score(y_test, y_test_pred_rf)

print("Random Forest - MSE Train:", mse_train_rf)
print("Random Forest - MSE TEST:", mse_test_rf)
print("Random Forest - R2 Train:", r2_train_rf)
print("Random Forest - R2 TEST:", r2_test_rf)

Random Forest - MSE Train: 6.292490479880424
Random Forest - MSE TEST: 33.33894626492995
Random Forest - R2 Train: 0.8658622681656934
Random Forest - R2 TEST: -0.4210338205773101


In [757]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

rf_model = RandomForestRegressor(n_estimators = 100, bootstrap=False, max_features = 'sqrt', random_state = 0)
rf_model.fit(X_train, y_train)

# Training set predictions and evaluation
y_train_pred_rf = rf_model.predict(X_train)
mse_train_rf = mean_squared_error(y_train, y_train_pred_rf)
r2_train_rf = r2_score(y_train, y_train_pred_rf)

# Test set predictions and evaluation
y_test_pred_rf = rf_model.predict(X_test)
mse_test_rf = mean_squared_error(y_test, y_test_pred_rf)
r2_test_rf = r2_score(y_test, y_test_pred_rf)

print("Hyperparameter Random Forest - MSE Train:", mse_train_rf)
print("Hyperparameter Random Forest - MSE TEST:", mse_test_rf)
print("Hyperparameter Random Forest - R2 Train:", r2_train_rf)
print("Hyperparameter Random Forest - R2 TEST:", r2_test_rf)

Hyperparameter Random Forest - MSE Train: 0.00510204081632653
Hyperparameter Random Forest - MSE TEST: 35.28099169886008
Hyperparameter Random Forest - R2 Train: 0.999891239218396
Hyperparameter Random Forest - R2 TEST: -0.5038112491373559


**k Nearest Neighbors**

In [749]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor()
model.fit(X_train, y_train)
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

KNeighborsRegressor(n_jobs=1)

In [750]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

model = KNeighborsRegressor()

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions using the model
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE) and R-squared (R2) for the training set
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Calculate Mean Squared Error (MSE) and R-squared (R2) for the testing set
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Print the results
print(f"MSE Train: {mse_train}")
print(f"MSE TEST: {mse_test}")
print(f"R2 Train: {r2_train}")
print(f"R2 TEST: {r2_test}")

MSE Train: 32.94652586807365
MSE TEST: 31.2303981904506
R2 Train: 0.29767517870798754
R2 TEST: -0.3311594105603317


### **Conclusion**

In conclusion, among all the regression algorithms we have implemented, **Decision Tree** and **Random Forest** gave us the best performance. The Decision Tree model achieved a Train MSE of **0.005** and a Train R2 of **0.99**, suggesting almost excellent performance on the training data. However, its Test MSE of **109.92** and Test R2 of **-3.68** indicated overfitting and poor generalization to unseen data. Through hyperparameter tuning with setting **random_state** to **0** and **splitter** to **"random"**, we improved the Test MSE to **41.86** and Test R2 to **-0.78** while maintaining Train MSE (**0.005**) and Train R2 (**0.99**).

Similarly, the Random Forest model initially produced a Train MSE of **6.29** and a Train R2 of **0.86**, with Test MSE of **33.33** and Test R2 of **-0.42**. After hyperparameter tuning with setting **bootstrap** to **False**, **max_features** to **'sqrt'**, and **random_state** to **0**, we achieved a Train MSE of **0.005** and a Train R2 of **0.999**, with Test MSE of **35.28** and Test R2 of **-0.50**, again indicating an excellent performance on the training data while maintaining same performance for training data. Moreover, both of these tuned implementations outperformed the base case, which had a Train MSE of **1.58**, Test MSE of **108.83**, Train R2 of **0.96**, and Test R2 of **-3.63**.

It's noteworthy that across all implementations, we consistently obtained negative values for the Test R2 metric, indicating that our models struggled to generalize to unseen data. This suggests a need for further exploration and potentially more advanced techniques or feature engineering to improve the models' generalization performance.